In [ ]:
import scripts.framework as fr
import scripts.headrpp_alg as headrpp

In [ ]:
repr_sp = fr.Representation("./data/nodes_sp.csv", "./data/edges_sp.csv", maptype="sp")
repr_sp.plot_map(plot_nodes=True)

repr_nl = fr.Representation("./data/nodes_nl.csv", "./data/edges_nl.csv", maptype="nl")
repr_nl.plot_map(plot_nodes=True)

In [ ]:
%matplotlib notebook
from IPython.display import HTML
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig = plt.figure(figsize=repr_sp.figsize)
ax = plt.axes()

def animate(i):
    ax.clear()
    for j, edge in enumerate(repr_sp.edges):
        if j == i:
            color = "red"
            linewidth = 3
            caption = f"Highlighted edge: {j}"
        else:
            color = "black"
            linewidth = 1
        ax.plot(
            [repr_sp.nodes[edge.source].x, repr_sp.nodes[edge.target].x],
            [
                repr_sp.figlims[1] - repr_sp.nodes[edge.source].y,
                repr_sp.figlims[1] - repr_sp.nodes[edge.target].y,
            ],
            color=color,
            linewidth=linewidth,
        )
    ax.set_xlim(0, repr_sp.figlims[0])
    ax.set_ylim(0, repr_sp.figlims[1])
    ax.set_title(caption)

ani = animation.FuncAnimation(fig, animate, frames=len(repr_sp.edges), interval=1000)

# save the animation to a file
ani.save("./images/animation_sp.gif", fps=1)

In [ ]:
nr_generations = 200

pop_size_sg = 50
alg = headrpp.HEADRPP(repr_sp, nr_generations, 22, 434, pop_size_sg)
routes, avg_fitness, best_fitness = alg.run_algorithm(show_results=True, save_name="images/singapore.png")
repr_sp.plot_map(routes[0], plot_nodes=False)

pop_size_nl = 10
alg = headrpp.HEADRPP(repr_nl, nr_generations, 0, 110, pop_size_nl)
routes, avg_fitness, best_fitness = alg.run_algorithm(show_results=True, save_name="images/netherlands.png")
repr_nl.plot_map(routes[0], plot_nodes=False)